# Transfer Learning with TensorFlow Part 1: Feature Extraction

Transfer learning is leveraging a working model's exisiting architecture and learned patterns for our own problem.

The two main benefits:
1. Can leverage an existing neural network architecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture which already learned patterns on similar data to our own, then we can adapt those patterns to our own data.